In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
import geopandas as gpd
from keplergl import KeplerGl
import pprint

In [2]:
data_dir = os.path.join("..", "COVID-19", "csse_covid_19_data", "csse_covid_19_time_series")
os.listdir(data_dir)

['time_series_19-covid-Confirmed.csv',
 '.gitignore',
 'time_series_19-covid-Recovered.csv',
 'time_series_19-covid-Deaths.csv',
 'README.md']

In [3]:
# data exported from google spreadsheet to csv for the confirmed case sheet
df = pd.read_csv(os.path.join(data_dir, "time_series_19-covid-Confirmed.csv"))

In [4]:
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,50,53,59,70,75,82,114,147,177,212.0
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,511,581,639,639,701,773,839,825,878,889.0
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,150,160,178,178,200,212,226,243,266,313.0
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,117,129,149,149,197,238,428,566,673,790.0


In [5]:
info_col = ["Province/State", "Country/Region", "Lat", "Long"]

col_dayend = []
col_dayend_str = []
for col_str in df.columns:
    if not(col_str in info_col):
        col_obj = datetime.strptime(col_str, "%m/%d/%y") # parse 1/21/2020 10:00 PM to the format
        #print(col_obj.date())
        #print(col_obj)
        if col_obj.date() in col_dayend:
            col_dayend = col_dayend[:-1]
            col_dayend_str = col_dayend_str[:-1]
        col_dayend.append(col_obj.date())
        col_dayend_str.append(col_str)
        #print(col)
        #print(type(col))
#len(col_dayend)
print(col_dayend_str)

['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20']


In [6]:
info_col2 = [ c for c in info_col if not c=="First confirmed date in country (Est.)" ]

In [7]:
df2 = df[info_col2+col_dayend_str]
#df2 = df2.set_index(info_col2)
df2.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,50,53,59,70,75,82,114,147,177,212.0
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,511,581,639,639,701,773,839,825,878,889.0
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,150,160,178,178,200,212,226,243,266,313.0
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,117,129,149,149,197,238,428,566,673,790.0


In [8]:
#df2.to_csv("time_series_2019-ncov-Confirmed-simplified.csv", index_label='ind')

In [9]:
df3 = pd.melt(df2, id_vars=info_col2, value_vars=col_dayend_str, var_name='datetime', value_name='confirmed_case')
df3.head()

,Province/State,Country/Region,Lat,Long,datetime,confirmed_case
0,NaN,Thailand,15.0000,101.0000,1/22/20,2.0
1,NaN,Japan,36.0000,138.0000,1/22/20,2.0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0.0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0.0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0.0


In [10]:
df3 = df3.dropna(subset=["confirmed_case"])
df3.head()

,Province/State,Country/Region,Lat,Long,datetime,confirmed_case
0,NaN,Thailand,15.0000,101.0000,1/22/20,2.0
1,NaN,Japan,36.0000,138.0000,1/22/20,2.0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0.0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0.0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0.0


In [11]:
# this is to show the data using the time slider
dates = []
for d in df3.datetime.tolist():
    dt_obj = datetime.strptime(d, "%m/%d/%y") # parse 1/21/2020 10:00 PM to the format
    dates.append(dt_obj.strftime("%Y-%m-%d")+" 00:00")
df3['datetime'] = dates

In [12]:
df3.head()

,Province/State,Country/Region,Lat,Long,datetime,confirmed_case
0,NaN,Thailand,15.0000,101.0000,2020-01-22 00:00,2.0
1,NaN,Japan,36.0000,138.0000,2020-01-22 00:00,2.0
2,NaN,Singapore,1.2833,103.8333,2020-01-22 00:00,0.0
3,NaN,Nepal,28.1667,84.2500,2020-01-22 00:00,0.0
4,NaN,Malaysia,2.5000,112.5000,2020-01-22 00:00,0.0


In [13]:
print(list(set(df3["datetime"])))

['2020-03-14 00:00', '2020-02-28 00:00', '2020-03-10 00:00', '2020-03-08 00:00', '2020-02-03 00:00', '2020-03-13 00:00', '2020-01-29 00:00', '2020-02-01 00:00', '2020-02-25 00:00', '2020-03-12 00:00', '2020-02-05 00:00', '2020-03-05 00:00', '2020-01-22 00:00', '2020-02-06 00:00', '2020-01-24 00:00', '2020-02-16 00:00', '2020-02-20 00:00', '2020-03-15 00:00', '2020-02-18 00:00', '2020-01-28 00:00', '2020-02-17 00:00', '2020-02-14 00:00', '2020-02-26 00:00', '2020-02-07 00:00', '2020-02-12 00:00', '2020-02-24 00:00', '2020-02-29 00:00', '2020-02-21 00:00', '2020-03-06 00:00', '2020-03-01 00:00', '2020-01-25 00:00', '2020-01-23 00:00', '2020-01-26 00:00', '2020-02-02 00:00', '2020-02-13 00:00', '2020-02-04 00:00', '2020-02-23 00:00', '2020-03-03 00:00', '2020-03-07 00:00', '2020-03-09 00:00', '2020-03-11 00:00', '2020-01-30 00:00', '2020-03-04 00:00', '2020-02-22 00:00', '2020-02-19 00:00', '2020-02-27 00:00', '2020-03-16 00:00', '2020-02-08 00:00', '2020-03-18 00:00', '2020-02-09 00:00',

In [14]:
(1579651140000 - 1579737480000)

-86340000

In [15]:
day0 = (int(datetime.strptime(df3["datetime"].min(), "%Y-%m-%d %H:%M").timestamp() * 1000 ))
day0, 1579651200000

(1579622400000, 1579651200000)

In [16]:
day1 = (int((datetime.strptime(df3["datetime"].min(), "%Y-%m-%d %H:%M")+timedelta(days=1) + timedelta(hours=1)).timestamp() * 1000 ))#+ 60000))
day1, 1579737600000

(1579712400000, 1579737600000)

In [17]:
(day1-day0)/24/60/60

1041.6666666666667

In [18]:
#df3.to_csv("time_series_2019-ncov-Confirmed-simplified_long.csv", index_label='ind')

In [19]:
point_gdf = gpd.GeoDataFrame(df3, geometry=gpd.points_from_xy(df3["Long"], df3["Lat"]))

In [25]:
%run map_time_series.conf.py
# use the previous config as base
config

{'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': 'confirmed_case',
     'name': 'datetime',
     'type': 'timeRange',
     'value': [1579651140000, 1579737480000]}],
   'layers': [{'id': 'or7ukl4',
     'type': 'point',
     'config': {'dataId': 'confirmed_case',
      'label': 'confirmed_case',
      'color': [227, 26, 26],
      'columns': {'lat': 'Lat', 'lng': 'Long', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 2,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
  

In [29]:
map_1 = KeplerGl(height=900, config=config)
map_1.add_data(data=point_gdf, name="confirmed_case")

map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'confirmed_case', 'name': 'da…

In [30]:
# add the time slider to map, need to manually add this to the config, because the save_to_html won't recognize the filter part

map_1.config["config"]['visState']["filters"] = [{"dataId": 'confirmed_case', "name": "datetime", 
                                                      'type': 'timeRange',
                                                      'value': [1579651200000, 1579737600000], ## a day
                                                      'name': 'datetime'}]


In [31]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(map_1.config)

{   'config': {   'mapState': {   'bearing': 0,
                                  'dragRotate': False,
                                  'isSplit': False,
                                  'latitude': 12.055250000000001,
                                  'longitude': 14.95205,
                                  'pitch': 0,
                                  'zoom': 2},
                  'mapStyle': {   'mapStyles': {},
                                  'styleType': 'dark',
                                  'threeDBuildingColor': [   9.665468314072013,
                                                             17.18305478057247,
                                                             31.1442867897876],
                                  'topLayerGroups': {},
                                  'visibleLayerGroups': {   '3d building': False,
                                                            'border': False,
                                                            'building

In [32]:
#if modification is done, uncomment the following and save the conf again
#with open("map_time_series.conf.py", "w") as f:
#    f.write('config = {}'.format(map_1.config))

In [33]:
map_1.save_to_html(file_name="map_time_series.html", config=map_1.config, read_only=True)

Map saved to map_time_series.html!
